# Semivariogram module

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот модуль предназначен для работы с семивариограммами и позволяет строить как экспериментальные данные, так и теоретические модели. Это применяется, например, в методах геостатистики, таких как кригинг. Также подобные инструменты встроены в другой модуль Sharp, где они позволяют проводить все автоматические вычисления.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В модуле Semivariogram есть ряд функций, которые доступны для внешнего вызова. К ним относятся:
- [semivariogramExperiment](#semivariogramExperiment "semivariogramExperiment");
- [getParameters](#getParameters "getParameters");
- [getCoeff](#getCoeff "getCoeff");
- [semivariogramTheory](#semivariogramTheory "semivariogramTheory");

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Помимо этого представлен ряд вспомогательных функций, вызов которых невозможен снаружи, но при этом облегчают понимание вышеперечисленных функций:
- [noZero](#noZero "noZero");
- [conctructKernel](#conctructKernel "conctructKernel");
- [OLSpolinom](#OLSpolinom "OLSpolinom");
- [OLSexponent](#OLSexponent "OLSexponent");
- [rmse](#rmse "rmse");
- [powerModel](#powerModel "powerModel");
- [exponentModel](#exponentModel "exponentModel");
- [gaussModel](#gaussModel "gaussModel");
- [spherModel](#spherModel "spherModel");
- [powExpModel](#powExpModel "powExpModel");

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ниже в [приложении](#Functions "Functions") будет дано описание каждой из них со списком принимаемых параметров и [код](#Script "Script").

## List Square (LS)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;В отличие от стандартного метода наименьших квадратов (МНК) в этом модуле реализованы иные виды: полиномиальный и экспоненциальный. Рассмотрим основную суть каждого из них вкратце.

#### Polinomial List Squares

(подробнее с этим можно ознакомиться в книге [Elementary Linear Algebra (9th Edition) - Howard Anton e Chris Rorres](https://docs.google.com/file/d/0B2ADaHV810Pwd081SEZ4Y2RRaE0/view) на 695 странице.)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Пусть есть наблюдения в нескольких точках {$x_1, x_2, x_3 \ldots x_n$}, в которых получены некотрые значения {$y_1, y_2, y_3 \ldots y_n$}. Пусть эти две переменные связаны между собой соотношением вида:

$$
\begin{equation}
  y(x) = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \ldots + a_m x^m
\end{equation}
\qquad \qquad (1)
$$

Тогда при подстановке в это уравнение наших данных получим следующую систему уравнений:

$$
\begin{equation}
 \left\{
  \begin{array}{cc}
    y_1 = a_0 + a_1 x_1 + a_2 x_1^2 + a_3 x_1^3 + \ldots + a_m x_1^m \cr
    y_2 = a_0 + a_1 x_2 + a_2 x_2^2 + a_3 x_2^3 + \ldots + a_m x_2^m \cr
    y_3 = a_0 + a_1 x_3 + a_2 x_3^2 + a_3 x_3^3 + \ldots + a_m x_3^m \cr
    \cdots \cdots \cdots \cdots \cdots \cdots \cdots \cdots \cr
    y_n = a_0 + a_1 x_n + a_2 x_n^2 + a_3 x_n^3 + \ldots + a_m x_n^m \cr
  \end{array}
 \right.
\end{equation}
\qquad \qquad (2)
$$

или в матричном виде:

$$
\begin{equation}
  \mathbf{Y}=\mathbf{X} \times \mathbf{M}
\end{equation}
\qquad \qquad (3), \qquad где
$$

$$
\begin{equation}
  \mathbf{Y}
    =  
  \begin{bmatrix}
    y_1\\
    y_2\\
    y_3\\
    ...\\
    y_n\\
  \end{bmatrix}, \qquad
  \mathbf{X}
  =  
  \begin{bmatrix}
    1 & x_1 & x_1^2 & x_1^3 & \cdots & x_1^m\\
    1 & x_2 & x_2^2 & x_2^3 & \cdots & x_2^m\\
    1 & x_3 & x_3^2 & x_3^3 & \cdots & x_3^m\\
    \cdots & \cdots & \cdots & \cdots & \cdots & \cdots\\
    1 & x_n & x_n^2 & x_n^3 & \cdots & x_n^m\\
  \end{bmatrix}, \qquad
  \mathbf{M}
  =  
  \begin{bmatrix}
    a_0\\
    a_1\\
    a_2\\
    \cdots\\
    a_m\\
  \end{bmatrix}
\end{equation}
$$

откуда получаем искомые коэффициенты:

$$
\begin{equation}
  \mathbf{M}=\mathbf{X^*} \times \mathbf{Y}
\end{equation}
\qquad \qquad (4)
$$

Здесь $\mathbf{X^*}$ - псевдообратная матрица.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Если, например, какие-либо из коэффициентов отсутствуют, то необходимо заменить соответствующий столбец $\mathbf{M}$ на столбец 0, провести те же самые вычисления и в ответе заменить получившиеся коэффициенты тоже на 0.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Этот метод наименьших квадратов используется для анализа сферической, степенной и степенно-экспоненциальной моделей семивариограмм.

#### Exponential Least Squares

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Пусть есть наблюдения в нескольких точках {$x_1, x_2, x_3 \ldots x_n$}, в которых получены некотрые значения {$y_1, y_2, y_3 \ldots y_n$}. Пусть эти две переменные связаны между собой соотношением вида:
$$
\begin{equation}
  f(x) = c_0 ( 1 - e^{-x a_0} )
\end{equation}
\qquad \qquad (5)
$$

Используя основное условие МНК: 

$$
\begin{equation}
  \sum_{k=1}^n (f(x_k) - y_k)^2 - min
\end{equation}
\qquad \qquad (6)
$$

можно получить следующую нелинейную систему уравнений:

$$
\begin{equation}
  \left\{
    \begin{array}{cc}
      c_0 \sum_{k=1}^n e^{-2a_0x} -2 c_0 \sum_{k=1}^n e^{-a_0x} + c_0 n + \sum_{k=1}^n e^{-a_0x} y - \sum_{k=1}^n y = 0 \cr
      -c_0 \sum_{k=1}^n e^{-2a_0x} x + c_0 \sum_{k=1}^n e^{-a_0x} x - \sum_{k=1}^n e^{-a_0x} xy = 0 \cr
    \end{array}
  \right.
\end{equation}
\qquad \qquad (7)
$$

Для решения получившейся системы применим метод Ньютона (подробнее об этом методе можно прочитать [здесь](http://www.apmath.spbu.ru/ru/structure/depts/is/task4-2013.pdf)). В результате получим следующие необходимые выражения:

$$
\begin{equation}
  \mathbf{X}^{(k + 1)}=\mathbf{X}^{(k)} - [\mathbf{F}'(\mathbf{X}^{(k)})]^{-1} \times \mathbf{F}(\mathbf{X}^{(k)})
\end{equation}
\qquad \qquad (8), \qquad где 
$$

$
\begin{equation}
  \mathbf{X} = \begin{bmatrix}
                 c_0\\ 
                 a_0 
               \end{bmatrix}
\end{equation}
\qquad - \ столбец \ коэффициентов;
$

$
\begin{equation}
  \mathbf{F}(\mathbf{X}) = \begin{bmatrix}
                                 f_1(x)\\ 
                                 f_2(x) 
                               \end{bmatrix} 
\end{equation}
\qquad - \ столбец \ исходных \ уравнений;
$

$
\begin{equation}
  \begin{bmatrix}
    \frac{\partial f_1(x)}{\partial c_0} & \frac{\partial f_1(x)}{\partial a_0}\\ 
    \frac{\partial f_2(x)}{\partial c_0} & \frac{\partial f_2(x)}{\partial a_0} 
  \end{bmatrix}
\end{equation}
\qquad - \ якобиан;
$

Учитывая систему уравнений (8), получам:

$$
\begin{equation}
  \begin{matrix}
    f_1(x) = c_0 \sum_{k=1}^n e^{-2a_0x} - 2 c_0 \sum_{k=1}^n e^{-a_0x} + c_0 n + \sum_{k=1}^n e^{-a_0x} y - \sum_{k=1}^n y \\
    f_2(x) = -c_0 \sum_{k=1}^n e^{-2a_0x} x + c_0 \sum_{k=1}^n e^{-a_0x} x - \sum_{k=1}^n e^{-a_0x} xy
  \end{matrix}
\end{equation}
\qquad \qquad (9) 
$$

***

$$
\begin{equation}
  \begin{matrix}
    \frac{\partial f_1(x)}{\partial c_0} = \sum_{k=1}^n e^{-2a_0x} - 2 \sum_{k=1}^n e^{-a_0x} + n \\
    \frac{\partial f_1(x)}{\partial a_0} = -2 c_0 \sum_{k=1}^n e^{-2a_0x} x + 2 c_0 \sum_{k=1}^n e^{-a_0x} x - \sum_{k=1}^n e^{-a_0x} xy \\
    \frac{\partial f_2(x)}{\partial c_0} = -\sum_{k=1}^n e^{-2a_0x} x + \sum_{k=1}^n e^{-a_0x} x \\
    \frac{\partial f_2(x)}{\partial a_0} = 2 c_0 \sum_{k=1}^n e^{-2a_0x} x^2 - c_0 \sum_{k=1}^n e^{-a_0x} x^2 + \sum_{k=1}^n e^{-a_0x} x^2y
  \end{matrix}
\end{equation}
\qquad \qquad (10) 
$$


## Semivariogram models

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;После того, как построены экспериментальные графики вариограмм, необходимо перейти к теоретическим [моделям семивариограмм](https://support.sas.com/documentation/cdl/en/statug/67523/HTML/default/viewer.htm#statug_krige2d_details01.htm). В данном модуле реализованы следующие типы моделей:
### <font color="red">1. Power model.</font>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Это одна из самых простых моделей, которую можно выразить простой формулой:

$$
\begin{equation}
  y(h) = c_0 h^{a_0}
\end{equation}
\qquad \qquad (11)
$$

Для выбора наилучших значений коэффициентов для этой модели, естественно воспользоваться МНК. После логарифмирования уравнения (11), его можно привести к полиному степени 1:

$$
\begin{equation}
  y^*(h) = c_0^* + a_0 h^* 
\end{equation}
\qquad \qquad (12), \qquad где
$$

$$
\begin{equation}
  y^*(h) = \ln y(h); \qquad c_0^* = \ln c_0; \qquad h^* = \ln h;  
\end{equation}
$$

после чего использовать [Polinomial Least Squares](#Polinomial-List-Squares "Polinomial List Squares"). В результате получим коэффициенты, которые дадут кривую наиболее близкую к нашим данным с точки зрения среднеквадратической ошибки.

### <font color="red">2. Exponential model.</font>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Уравнение подобной модели выглядит как:

$$
\begin{equation}
  y(h) = c_0 [1 - e^{-\frac{h}{a_0}}]
\end{equation}
\qquad \qquad (13)
$$

Для подбора наилучших коэффициентов можно воспользоваться [Exponential Least Squares](#Exponential-Least-Squares "Exponential Least Squares"), где нужно заменить коэффициент $\mathbf{a_0}$ на $\frac{\mathbf{1}}{\mathbf{a_0}}$.

### <font color="red">3. Gaussian model.</font>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Уравнение подобной модели выглядит как:

$$
\begin{equation}
  y(h) = c_0 [1 - e^{-(\frac{a_0}{h})^2}]
\end{equation}
\qquad \qquad (14)
$$

Для подбора наилучших коэффициентов можно воспользоваться [Exponential Least Squares](#Exponential-Least-Squares "Exponential Least Squares"), где нужно заменить коэффициент $\mathbf{a_0}$ на $\frac{\mathbf{1}}{\mathbf{a_0^2}}$, а переменную $\mathbf{h}$ на $\mathbf{h^2}$.

### <font color="red">4. Spherical model.</font>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ее уравнение принимает вид:

$$
\begin{equation}
  \left\{\begin{matrix}
    c_0 [\frac{3}{2} \frac{h}{a_0} - \frac{1}{2} (\frac{h}{a_0})^3] & for \ h \leq a_0\\ 
    c_0 & for \ h > a_0
  \end{matrix}\right.
\end{equation}
\qquad \qquad (15)
$$

Для подбора наилучших коэффициентов можно воспользоваться [Polinomial Least Squares](#Polinomial-List-Squares "Polinomial List Squares"). При этом следует помнить, что на одном интервале полином третьей степени с нулевыми коэффициентами при свободном члене и неизвестной второй степени, а на втором - полином представляет из себя только свободный член. Чтобы реализовать эту модель, нужно воспользоваться [Методами оптимизации](http://www.lib.uniyar.ac.ru/edocs/iuni/20150201.pdf). Но для простоты можно поступить иначе:
- разбить весь набор данных на две группы во всевозможных вариантах; при этом в каждую группу должны входить последовательно идущие точки и граничная точка входит в обе группы!;
- применить к первой группе только первую часть сферической модели (то есть полином третьей степени); при этом ограничения и вторую часть модели игнорируем;
- найти при этих условиях значения коэффициентов $\mathbf{a_0}$ и $\mathbf{c_0}$;
- если, например, точкой разделения служит $\mathbf{x_i}$, то из всех вариантов, полученных на предыдущем шаге, оставить только те, которые удовлетворяют неравенству: $\mathbf{x_i} < \mathbf{a_0} < \mathbf{x_{i + 1}}$; это неравенство показывает, что данная кривая не выйдет за пределы текущей и последующей точек (иначе наборы точек будут изменены);
- для оставшихся вариантов посчитать СКО как для первой части, так и для второй части модели;
- выбрать тот вариант, который даст минимальную СКО.

### <font color="red">5. Power-exponential model.</font>  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Это комбинация степенной и экспоненциальной модели. В соответствии с этим уравнение принимает вид:

$$
\begin{equation}
  y(h) = c_0 [1 - e^{-\frac{h}{a_0}}] + d_0 h^{b_0}
\end{equation}
\qquad \qquad (16)
$$

Здесь также, используя суть метода МНК, можно построить систему уравнений и решить ее одним из численных методов. Однако наилучшим решением в этом случае будет лишь метод Ньютона (в виду сложности системы). Как при этом показал опыт, сходимость при этом не всегда гарантируется. Даже подбор близких к решению начальных значений не всегда приводит к жедаемому результату. Поэтому поступим по-другому:
- подбор наиболее подходящей [Power model](#1.-Power-model. "Power model");
- подбор наиболее подходящей [Exponential model](#2.-Exponential-model. "Exponential model");
- введение нового коэффициента $\mathbf{\alpha}$ и решение методом МНК уравнения:

$$
\begin{equation}
  y(h) = \alpha [c_0 [1 - e^{-\frac{h}{a_0}}]] + (1 - \alpha) [d_0 h^{b_0}]
\end{equation}
\qquad \qquad (17)
$$

 которое легко сводится к решению уравнения вида 
 $
 \begin{equation}
   \mathbf{A} \times \mathbf{X} = \mathbf{Y}
 \end{equation}
 .
 $
 
Иными словами сначала подбираются и фиксируются коэффициенты $\mathbf{a_0}$ и $\mathbf{b_0}$. После этого подстраиваем и коэффициенты $\mathbf{c_0}$ и $\mathbf{d_0}$. При таком подходе получим кривую, которая не будет давать минимальную ошибку. Однако она будет очень близка к этому. К тому же это не играет никакой роли, поскольку в большинстве случаев она будет описывать ситуацию значительно лучше, нежели любая из выше описанных моделей.

---

## Functions

### semivariogramExperiment 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get experiment variance point using image. You can set some sattings to control the interval of points in which all calculations will be done. Use also one of the kernel type if you are interested in considering direcions or not.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-kfkg{background-color:#dae8fc;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-n533{font-weight:bold;background-color:#dae8fc;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">image</td>
    <td class="tg-c3ow">{image}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Image using for calculating semivariograms. Note that for the convolution you can use only standart types of kernel either considering directions or not. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-n533">SrartPoint</td>
    <td class="tg-kfkg">{number}</td>
    <td class="tg-kfkg"><span style="font-style:italic">Initial point, which using for calculating convolution. This number corresponds to which lag would be the first. For example,</span>  <span style="font-weight:bold">StartPoint = 3 </span><span style="font-weight:normal;font-style:italic">with kernel</span><span style="font-weight:normal"> </span><span style="font-weight:bold">type = 'Smooth'</span><span style="font-weight:normal"> </span><span style="font-weight:normal;font-style:italic">return first lag = 60 m.</span><span style="font-weight:normal"> </span><span style="font-weight:normal;color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">StartPoint = 0</span><span style="font-weight:normal">.</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">EndPoint</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Terminal point, which using for calculating convolution. This number corresponds to which lag would be the last. </span><span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">EndPoint = 15</span><span style="font-style:italic">.</span></td>
  </tr>
  <tr>
    <td class="tg-n533">geometry</td>
    <td class="tg-kfkg">{geometry}</td>
    <td class="tg-kfkg"><span style="font-style:italic">Geometry object in which bounds calculations will be done.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">geometry = 'auto</span><span style="font-weight:bold">'</span> <span style="font-style:italic;text-decoration:underline">(this means that polygon with image size will be created)</span><span style="font-style:italic">.</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">scale</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The scale of the image which using for variance calculation.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">scale = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that the scale of current image will be used).</span></td>
  </tr>
  <tr>
    <td class="tg-n533">proj</td>
    <td class="tg-kfkg">{projection}</td>
    <td class="tg-kfkg"><span style="font-style:italic">The projection of the image which using in variance calculation.</span> <span style="color:rgb(50, 203, 0)">Default:</span><span style="font-weight:bold;color:rgb(50, 203, 0)"> proj = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that the projection of current image will be used).</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">type</td>
    <td class="tg-c3ow">{String}</td>
    <td class="tg-c3ow"><span style="font-style:italic">Type of the kernel using in the convolution. You can use only 5 types: </span><span style="font-weight:bold;font-style:italic">'N-S'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'W-E'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'NE-SW'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'NW-SE'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'Smooth'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">type = 'Smooth'</span>.</td>
  </tr>
</table>

##### Return: list of two lists. The first is the list of lags and the second is corresponding list of semivariograms.
#### Return: list

***

### getParameters

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Just get spherical model's coefficients for current observed set.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of two variables's lists. First list is independent and the second is dependent.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: coefficients of spherical model.
#### Return: list

***

### getCoeff

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate coefficients for using model. You can do it automatically or using your settings.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of two variables' lists. First list is independent and the second is dependent.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">The name of using model. Available names are: </span><span style="font-weight:bold;font-style:italic">'power'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'spher'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
  <tr>
    <td class="tg-amwm">iterate</td>
    <td class="tg-baqh">{number}</td>
    <td class="tg-baqh"><span style="font-style:italic">The number of repeating iterating process for calculating coefficients. Note this parameter is required if only you use following models: </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-weight:normal;font-style:italic">,</span><span style="font-style:italic"> </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">iterate = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that it would be 50 or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">initValues</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of init values for iterating process. Note this parameter is required if only you use following models: </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss</span><span style="font-style:italic">', </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">initValues = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that they would be calculated automatically or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
</table>

##### Return: coefficients of selecting model.
#### Return: list

***

### semivariogramTheory

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get semivariogram value using one of semivariogram models.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">lag</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">x-value; it could be lag or another independent value.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">model</td>
    <td class="tg-baqh">{String}</td>
    <td class="tg-baqh"><span style="font-style:italic">The name of using model. Availabke names are: </span><span style="font-weight:bold;font-style:italic">'power'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'exponent'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'gauss'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'spher'</span><span style="font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-style:italic">.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">model = 'powExp'</span>.</td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff</td>
    <td class="tg-baqh">{list}</td>
    <td class="tg-baqh"><span style="font-style:italic">The list of coefficients for using model. Note this parameter should contains only 2 parameters for all models except </span><span style="font-weight:bold;font-style:italic">'powExp'</span><span style="font-weight:normal;font-style:italic">: in this case use 4 parameters.</span> <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">coeff = 'auto'</span> <span style="font-style:italic;text-decoration:underline">(this means that they would be all 1 or </span><span style="font-weight:bold;font-style:italic;text-decoration:underline">null</span><span style="font-style:italic;text-decoration:underline"> depending on the model).</span></td>
  </tr>
</table>

##### Return: value of selecting model's semivariogram function.
#### Return: number

***

### noZero

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Exclude all zero values from observed data. Note that it is cinsidered the data has format of list which contains two sublists; they should have equal size. Each value from first list with corresponding value from second list are pair. If only one of values within pair is zero, whole pair will be exclude.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of two variables' lists First list is independent and the second is dependent.</span> <span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: list of data that do not contain zero value.
#### Return: list

***

### conctructKernel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Construct kernel which will be using to calulate experimental semivariogram points.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">type</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The type of using kernel. Available types are: </span><span style="font-weight:bold;font-style:italic">'N-S'</span><span style="font-weight:normal;font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'W-E'</span><span style="font-weight:normal;font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'NE-SW'</span><span style="font-weight:normal;font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'NW-SE'</span><span style="font-weight:normal;font-style:italic">, </span><span style="font-weight:bold;font-style:italic">'Smooth'</span><span style="font-weight:normal;font-style:italic">. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">semisise</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">The half of kernel size. If kernel has odd number of length, take this number and round it after division by 2. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">d1</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">The distance between 2 nearest pixel centroids which do not lie on the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">d2</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">The distance between 2 nearest pixel centroids which lie on the diagonal. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff_central</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">The weight which will be placed in kernel's centre. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff_out_direct</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">Weights which are placed in kernel if you want to consider directions. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-amwm">coeff_out_smooth</td>
    <td class="tg-yw4l">{number}</td>
    <td class="tg-yw4l"><span style="font-style:italic">Weights which are placed in kernel if you want to not consider directions. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: kernel for following experimental semivariograms calculation.
#### Return: kernel

***

### OLSpolinom

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculation Least Square for approximating polinom. You can set some settings such using polinom with zero coefficients.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observedSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of two variables' lists. First list is independent and the second is dependent</span><span style="font-weight:normal;font-style:italic">. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">PowerPolinom</td>
    <td class="tg-us36">{number}</td>
    <td class="tg-us36"><span style="font-style:italic">The power of approximating polinom. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">zeroCoeff</td>
    <td class="tg-us36">{number}</td>
    <td class="tg-us36">The list contains place number for zero coefficients. Count begins with zero power. <span style="color:rgb(50, 203, 0)">Default: </span><span style="font-weight:bold;color:rgb(50, 203, 0)">zeroCoeff = 'null'</span>.</td>
  </tr>
</table>

##### Return: list of coefficients for approximating polinom.
#### Return: list

***

### OLSexponent

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculation Least Square for approximating exponent function. This requires some initial data such as iterate number and init values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-us36{border-color:inherit;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observedSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of two variables' lists. First list is independent and the second is dependent</span><span style="font-weight:normal;font-style:italic">. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">iterate</td>
    <td class="tg-us36">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of  repeating iterate process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">initValues</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of init values for iterating process. Note that this list should contain all required values. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: list of coefficients for approximating exponent model.
#### Return: list

***

### rmse

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get root mean square error using experimental and theoretical data.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of experimental data. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">modelSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of data getting from theoretical function. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: root mean square error for using data.
#### Return: number

***

### powerModel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for power semivariogram model using observe set. This list conclude 2 lists of independent and dependent variables. Note that they should be in pairs such {x,y}.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of 2 variables' lists. First list is independent and the second is dependent. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: the list of power model's coefficients.
#### Return: list

***

### exponentModel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for exponent semivariogram model using observe set. This list conclude 2 lists of independent and dependent variables. Note that they should be in pairs such {x,y}. For calculating use iterate number and correct init values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of 2 variables' lists. First list is independent and the second is dependent. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">iterate</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of repeating iterate process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">initValues</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of init values for iterating process. Note the list should contain 2 values for 2 correspond coefficients. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: the list of exponent model's coefficients.
#### Return: list

***

### gaussModel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for gaussian semivariogram model using observe set. This list conclude 2 lists of independent and dependent variables. Note that they should be in pairs such {x,y}. For calculating use iterate number and correct init values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of 2 variables' lists. First list is independent and the second is dependent. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">iterate</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of repeating iterate process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">initValues</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of init values for iterating process. Note the list should contain 2 values for 2 correspond coefficients. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: the list of gaussian model's coefficients.
#### Return: list

***

### spherModel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for spherical semivariogram model using observe set. This list conclude 2 lists of independent and dependent variables. Note that they should be in pairs such {x,y}.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of 2 variables' lists. First list is independent and the second is dependent. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: the list of spherical model's coefficients.
#### Return: list

***

### powExpModel

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Get coefficients for power-exponential semivariogram model using observe set. This list conclude 2 lists of independent and dependent variables. Note that they should be in pairs such {x,y}. For calculating use iterate number and correct init values.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-2dfk{font-weight:bold;background-color:#ecf4ff;border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-7btt{font-weight:bold;border-color:inherit;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-2dfk">Parameteres</th>
    <th class="tg-2dfk">Type</th>
    <th class="tg-2dfk">Description</th>
  </tr>
  <tr>
    <td class="tg-7btt">observeSet</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-weight:normal;font-style:italic">The list of 2 variables' lists. First list is independent and the second is dependent. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">iterate</td>
    <td class="tg-c3ow">{number}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The number of repeating iterate process. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
  <tr>
    <td class="tg-7btt">initValues</td>
    <td class="tg-c3ow">{list}</td>
    <td class="tg-c3ow"><span style="font-style:italic">The list of init values for iterating process. Note the list should contain 4 values for 4 correspond coefficients. </span><span style="font-style:italic;color:rgb(254, 0, 0)">This parameter is required!</span></td>
  </tr>
</table>

##### Return: the list of power-exponential model's coefficients.
#### Return: list

***

## Script

In [ ]:
// semivariogramExperiment: constructing experimental semivariogram;

// image {image} - the image using for building semivariogram;
// SrartPoint, EndPoint {number} - the number of initial and terminal points respectively (count starts with 0!);
// SrartPoint - (default: 0) ; EndPoint - (default: 15)
// geometry {geometry} - geometry object in which the image locates {default: all image};
// scale {number} - the scale of the image (default: nominal scale of image);
// proj {projection} - the project of the image (default: projection of giving image);
// type {string} - type of the Kernel using in convolution (default: 'Smooth').

// return: dictionary of two lists: distances and semivariograms; {dictionary[lags, semivar]}; 

exports.semivariogramExperiment = function (image, SrartPoint, EndPoint, geometry, scale, proj, type) {
  
  geometry = ee.Algorithms.If(ee.Algorithms.IsEqual(geometry, null), ee.Feature(image).geometry(), geometry);
  SrartPoint = ee.Algorithms.If(ee.Algorithms.IsEqual(SrartPoint, null), 0, SrartPoint);
  EndPoint = ee.Algorithms.If(ee.Algorithms.IsEqual(EndPoint, null), 15, EndPoint);
  scale = ee.Algorithms.If(ee.Algorithms.IsEqual(scale, null), image.projection().nominalScale(), scale);
  proj = ee.Algorithms.If(ee.Algorithms.IsEqual(proj, null), image.projection(), proj);
  type = ee.Algorithms.If(ee.Algorithms.IsEqual(type, null), 'Smooth', type);
  type = ee.Algorithms.String(type);
  
  var sequence = ee.List.sequence(SrartPoint, ee.Number(EndPoint).subtract(1));
  
  var coeff_central = -1;
  var coeff_out_direct = 1;
  var coeff_out_smooth = ee.Number(1).divide(4);
  
  var lags = sequence.iterate (function (i, list) {
    
    var index1 = ee.Number(ee.List(ee.List(list).get(0)).get(ee.Number(i)));
    var index2 = ee.Number(ee.List(ee.List(list).get(1)).get(ee.Number(i)));
    
    var d1 = (index1.add(1)).multiply(scale);
    var d2 = ee.Number(ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'Smooth'),
             index2.add(1).multiply(scale).multiply(ee.Number(2).sqrt()), 
             index2.add(1).multiply(scale)));
    
    var d = d2.min(d1);
    
    var index1_new = ee.Algorithms.If(d1.lte(d2), index1.add(1), index1);
    var index2_new = ee.Algorithms.If(d2.lte(d1), index2.add(1), index2);
   
    var j = ee.Number(index1_new).max(index2_new);
    
    var Kernel = conctructKernel(type, j, d1, d2, coeff_central, coeff_out_direct, coeff_out_smooth);

    var convolved = image.convolve(Kernel);

    var lag = convolved.reduceRegion({
      reducer: ee.Reducer.variance(),
      geometry: geometry,
      scale: scale,
      crs: proj,
      bestEffort: true
    });
    
    lag = lag.get(ee.List(lag.keys()).get(0));
    lag = ee.Number(lag).divide(2);
    
    var list1 = ee.List(ee.List(list).get(0)).add(index1_new);
    var list2 = ee.List(ee.List(list).get(1)).add(index2_new);
    var list3 = ee.List(ee.List(list).get(2)).add(d);
    var list4 = ee.List(ee.List(list).get(3)).add(lag);
    
    return ee.List([list1, list2, list3, list4]);
  }, ee.List([[0], [0], [0], [0]]));

  var distance = ee.List(lags).get(2);
  var lag = ee.List(lags).get(3);

  return ee.List([distance, lag]);
  
};


// getParameters: finding sill and range;

// observeSet {list} - list of observing variable;

// return: sill and range; {list};

exports.getParameters = function (observeSet) {
  return spherModel(observeSet);
};


// getCoeff: get coefficients for semivariogram using observed data;

// observeSet {list} - list of observing variable;
// model {string} - the name of using model (default: 'powExp');
// iterate {number}  - counts of repeating iterating process (default: '50');
// initValues {object} - initial values for iterating process (default: 'auto');

// return: list of coefficients for this model; {list};

exports.getCoeff = function (observeSet, model, iterate, initValues) {
  
  model = ee.Algorithms.If(ee.Algorithms.IsEqual(model, null), 'powExp', model);
  iterate = ee.Algorithms.If(ee.List(['exponent', 'gauss', 'powExp']).contains(model),
            ee.Algorithms.If(ee.Algorithms.IsEqual(iterate, null), 50, iterate), null);

  initValues = ee.Algorithms.If(ee.Algorithms.IsEqual(initValues, null), 
               ee.List(spherModel(observeSet)).slice(0, 2), initValues);
  
  var coeff =  ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'power'), powerModel (observeSet),
               ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'exponent'), exponentModel (observeSet, iterate, initValues),
               ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'gauss'), gaussModel (observeSet, iterate, initValues), 
               ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'spher'), spherModel (observeSet), 
               ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'), powExpModel (observeSet, iterate, initValues), 
               null)))));
  
  return coeff;
            
};  


// semivariogramTheory: construct theoretical semivariogram using observed data;

// lag {list} - list of observing variable;
// model {string} - the name of using model (default: 'powExp');
// coeff {object} - model's coefficients for constructing semivariogram (default: all 1);

// return: value of semivariograms for using model; {number};

exports.semivariogramTheory = function (lag, model, coeff) {
  
  coeff = ee.Algorithms.If(ee.Algorithms.IsEqual(coeff, null), ee.List([1, 1, 1, 1]), coeff);
  
  model = ee.Algorithms.If(ee.Algorithms.IsEqual(model, null), 'powExp', model);
  model = ee.Algorithms.String(model);
  
  var c0 = ee.Number(ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'),
           ee.List(coeff).get(2), ee.List(coeff).get(0)));
  var a0 = ee.Number(ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'),
           ee.List(coeff).get(0), ee.List(coeff).get(1)));
  var b0 = ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'),
           ee.List(coeff).get(1), null);
  var d0 = ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'),
           ee.List(coeff).get(3), null);
  
  var powerFunction = function (h) {
    h = ee.Number(h);
    return h.pow(a0).multiply(c0);
  };
  
  var sphericalFunction = function (h) {
    h = ee.Number(h);
    return ee.Algorithms.If(h.lte(a0), 
    c0.multiply(h.divide(a0).multiply(3).divide(2).subtract(h.divide(a0).pow(3).divide(2))),
    c0);
  };
    
  var exponentialFunction = function (h) {
    h = ee.Number(h);
    return c0.multiply(ee.Number(1).subtract(h.divide(a0).multiply(-1).exp()));
  };
  
  var gaussianFunction = function (h) {
    h = ee.Number(h);
    return c0.multiply(ee.Number(1).subtract(h.divide(a0).pow(2).multiply(-1).exp()));
  };
  
  var powExpFunction = function (h) {
    h = ee.Number(h);
    b0 = ee.Number(b0);
    d0 = ee.Number(d0);
    return c0.multiply(ee.Number(1).subtract(h.divide(a0).multiply(-1).exp()))
           .add(h.pow(b0).multiply(d0));
  };
  
  var semivar_value = function (lag) {
    return  ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'power'), powerFunction (lag),
            ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'spher'), sphericalFunction (lag),
            ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'exponent'), exponentialFunction (lag),
            ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'gauss'), gaussianFunction (lag),
            ee.Algorithms.If(ee.Algorithms.IsEqual(model, 'powExp'), powExpFunction (lag), 
           'Do not found model')))));
  };
  
  return semivar_value(lag);
  
};


// noZero: exclude zero values from observe set (excluding both X and Y if one of them is zero);

// observedSset {list} - observation set: list of two lists: independent set and dependent set;

// return: non zero set; {list};

var noZero = function(observeSet) {
  
  var list = ee.List.sequence(0, ee.Number(ee.List(ee.List(observeSet).get(0)).length()).subtract(1));
  
  var observeSet_NoZero = list.map(function (i) {
    
    i = ee.Number(i);
    var X = ee.List(observeSet).get(0);
    var Y = ee.List(observeSet).get(1);
    var elementX = ee.Number(ee.List(X).get(i));
    var elementY = ee.Number(ee.List(Y).get(i));
    
    return ee.Algorithms.If(elementX.eq(0).or(elementY.eq(0)),
           ee.List([null, null]), ee.List([elementX, elementY]));
    
  });
  
  observeSet_NoZero = ee.Array(observeSet_NoZero.removeAll([[null, null]])).matrixTranspose(0, 1).toList();
  
  return observeSet_NoZero;
  
};


// conctructKernel: construct kernel;

// list of parametres;

// return: Kernel; {kernel};

var conctructKernel = function (type, semisise, d1, d2, coeff_central, coeff_out_direct, coeff_out_smooth) {
    
  semisise = ee.Number(semisise);
  d1 = ee.Number(d1);
  d2 = ee.Number(d2);
    
  var row = ee.List.repeat(0, semisise.multiply(2).add(1));
  var box = ee.List.repeat(row, semisise.multiply(2).add(1));
  var row_center = row.set(semisise, coeff_central);
  box = box.set(semisise, row_center);
    
  var row1 = row.set(0, coeff_out_smooth).set(-1, coeff_out_smooth);
  var row_center1 = row_center;
  var row_center2 = row_center.set(0, coeff_out_smooth).set(-1, coeff_out_smooth);
  var row2 = row.set(semisise, coeff_out_smooth);
    
  var box1 = box.set(0, row1).set(semisise, row_center1).set(-1, row1);
  var box2 = box.set(0, row2).set(semisise, row_center2).set(-1, row2);
    
  box = 
    ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'N-S'), box.set(0, row.set(semisise, coeff_out_direct)), 
    ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'W-E'), box.set(semisise, row_center.set(0, coeff_out_direct)), 
    ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'NE-SW'), box.set(0, row.set(-1, coeff_out_direct)),
    ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'NW-SE'), box.set(0, row.set(0, coeff_out_direct)), 
    ee.Algorithms.If(ee.Algorithms.IsEqual(type, 'Smooth'),
    ee.Algorithms.If(d2.lt(d1), box1, box2))))));
      
  var Kernel = ee.Kernel.fixed ({
    width: semisise.multiply(2).add(1),
    height: semisise.multiply(2).add(1),
    weights: box,
    x: semisise.multiply(-1),
    y: semisise.multiply(-1),
    normalize: false
  });  
    
  return Kernel;
    
};


// OLSpolinom: using linear algebra to construct OLS equations using polinoms as approximation;
// y (x) = a0 + a1*x + a2*x^2 + a3*x^3 + ...;

// observedSet {list} - observation set: list of two lists: independent set and dependent set;
// PowerPolinom {number} - maximum power of using polinom;
// zeroCoeff  {list} - list of coefficients' number which value should be zero (count starts with a0 as number 0 coefficient!);

// return: list of coefficients such as [a0, a1, a2, a3, ...]; {list};

var OLSpolinom = function (observedSet, PowerPolinom, zeroCoeff) {
  
  var observed_x = ee.List(observedSet).get(0);
  var observed_y = ee.List(observedSet).get(1);
  
  var vector_y = ee.List(observed_y);
  var add_y = ee.List.repeat(0, vector_y.length());
  
  var Matrix_y = ee.List.repeat(add_y, ee.Number(PowerPolinom).add(1)); 
  Matrix_y = Matrix_y.set(0, vector_y); 
  
  var List_x = ee.List.repeat(observed_x, ee.Number(PowerPolinom).add(1));
  var List_pow = ee.List.sequence(0, PowerPolinom);
  
  var Matrix_M = List_pow.map(function (col) {
    return ee.Array(List_x.get(col)).pow(col).toList();
  });
  
  var Matrix__M = ee.Algorithms.If(ee.List(zeroCoeff).containsAll([null]), 
  Matrix_M, ee.List(zeroCoeff).iterate(function (i, Matrix_M_) {
    var col = ee.List.repeat(0, ee.List(observed_x).length());
   return  ee.List(Matrix_M_).set(ee.Number(i), col);
  }, Matrix_M));

  Matrix__M = ee.Array(Matrix__M).matrixTranspose(0, 1);
  
  var M_pdeudoinverse = ee.Array(Matrix__M).matrixPseudoInverse();
  
  var coeff = M_pdeudoinverse.matrixMultiply(ee.Array(Matrix_y).matrixTranspose(0, 1)).matrixTranspose(1, 0);
  coeff = coeff.toList().get(0);
  
  var coeff_ = ee.Algorithms.If(ee.List(zeroCoeff).containsAll([null]), 
  coeff, ee.List(zeroCoeff).iterate(function (i, coeff) {
    return ee.List(coeff).set(i, 0);
  }, coeff));
  
  return coeff_;
  
};


// OLSexponent: using linear algebra to construct OLS equations using exponent model as approximation;
// y (x) = a - a*exp(-x*b);

// observedSet {list} - observation set: list of two lists: independent set and dependent set;
// iterate {number}  - counts of repeating iterating process;
// initValues {list} - initial values for iterating process;

// return: list of coefficients such as [a, b]; {list};


var OLSexponent = function (observedSet, iterate, initValues) {
  
  var x = ee.Array(ee.List(observedSet).get(0));
  var y = ee.Array(ee.List(observedSet).get(1));
  
  var sequence = ee.List.sequence(0, iterate);
  
  var coeff = sequence.iterate(function (n, list) {
    
    var a = ee.Number(ee.List(list).get(0));
    var b = ee.Number(ee.List(list).get(1));
    
    var sum1 = x.multiply(-2).multiply(b).exp().reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum2 = x.multiply(-1).multiply(b).exp().reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum3 = x.multiply(-1).multiply(b).exp().multiply(y).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum4 = x.length().get([0]);
    var sum5 = y.reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum6 = x.multiply(-2).multiply(b).exp().multiply(x).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum7 = x.multiply(-1).multiply(b).exp().multiply(x).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum8 = x.multiply(-1).multiply(b).exp().multiply(x).multiply(y).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum9 = x.multiply(-2).multiply(b).exp().multiply(x.pow(2)).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum10 = x.multiply(-1).multiply(b).exp().multiply(x.pow(2)).reduce(ee.Reducer.sum(), [0]).get([0]);
    var sum11 = x.multiply(-1).multiply(b).exp().multiply(x.pow(2)).multiply(y).reduce(ee.Reducer.sum(), [0]).get([0]);
    
    var f1 = sum1.multiply(a).subtract(sum2.multiply(a).multiply(2)).add(sum4.multiply(a)).add(sum3).subtract(sum5);
    var f2 = sum7.multiply(a).subtract(sum6.multiply(a)).subtract(sum8);
    
    var Matrix_F = ee.Array([[f1, 0],
                             [f2, 0]]);    
    
    var f1_derivation_a = sum1.subtract(sum2.multiply(a)).add(sum4);
    var f1_derivation_b = sum7.multiply(a).multiply(2).subtract(sum6.multiply(a).multiply(2)).subtract(sum8);
    var f2_derivation_a = sum7.subtract(sum6);
    var f2_derivation_b = sum9.multiply(a).multiply(2).subtract(sum10.multiply(a)).add(sum11);
    
    var Jacobian = ee.Array([[f1_derivation_a, f1_derivation_b],
                             [f2_derivation_a, f2_derivation_b]]);
    
    var Jacobian_inverse = Jacobian.matrixInverse();                     
    
    var coeff = ee.Array(list);
    var coeff_new =  coeff.subtract(ee.Array(Jacobian_inverse.matrixMultiply(Matrix_F).matrixTranspose(0, 1).toList().get(0))).toList();
    
    return coeff_new;
    
  }, initValues);
  
  return coeff;
  
};

// Root Mean Square Error (rmse): calculate error between actual values and model;

// observeSet {list} - list of observing variable;
// modelSet {list}  - list of modeling variable;

// return: error; {number};

var rmse = function (observeSet, modelSet) {
  var difference = ee.Array(observeSet).subtract(ee.Array(modelSet)).pow(2).toList();
  return ee.Number(difference.reduce(ee.Reducer.sum())).divide(ee.Number(difference.length()).subtract(1)).sqrt();
};


// Constructing theoretical variograms:
  
// a) powerModel: The Power Semivariogram Model: y(h) = c0*h^(a0);

// observeSet {list} - list of observing variable;

// return: Dictionary of coefficients for this model; {Dictionary};


var powerModel = function (observeSet) {
  
  var PowerPolinom = 1;
  var zeroCoeff = ee.List([null]);
  
  observeSet = noZero(observeSet); 
  
  var observeX = ee.Array(ee.List(ee.List(observeSet).get(0))).log().toList();
  var observeY = ee.Array(ee.List(ee.List(observeSet).get(1))).log().toList();
  
  observeSet = ee.List([observeX, observeY]);
  
  var coeff = OLSpolinom(observeSet, PowerPolinom, zeroCoeff);
  
  coeff = ee.List(coeff).set(0, ee.Number(ee.List(coeff).get(0)).exp());
  
  var f_model = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return ee.Number(ee.List(coeff).get(0)).multiply(ee.Number(x).pow(ee.List(coeff).get(1)));
  });
  
  var error = rmse(ee.List(observeSet).get(1), f_model);
  coeff = ee.List(coeff).add(error);

  return coeff;

};


// b) exponentModel: The Exponential Semivariogram Model: y(h) = c0*[1 - exp(-h / a0)];

// observeSet {list} - list of observing variable;
// iterate {number}  - counts of repeating iterating process;
// initValues {list} - initial values for iterating process;

// return: Dictionary of coefficients for this model; {Dictionary};

var exponentModel = function (observeSet, iterate, initValues) {
  
  initValues = ee.List(initValues).set(1, ee.Number(1).divide(ee.Number(ee.List(initValues).get(1))));
  
  var coeff = OLSexponent(observeSet, iterate, initValues);
  
  coeff = ee.List(coeff).set(1, ee.Number(ee.List(coeff).get(1)).pow(-1));
  
  var f_model = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return ee.Number(ee.List(coeff).get(0)).multiply(ee.Number(1).subtract((ee.Number(x).multiply(-1).divide(ee.List(coeff).get(1))).exp()));
  });
  
  var error = rmse(ee.List(observeSet).get(1), f_model);
  coeff = ee.List(coeff).add(error);

  return coeff;

};


// c) gaussModel: The Gaussian Semivariogram Model: y(h) = c0*[1 - exp(-(h / a0)^2)];

// observeSet {list} - list of observing variable;
// iterate {number}  - counts of repeating iterating process;
// initValues {list} - initial values for iterating process;

// return: Dictionary of coefficients for this model; {Dictionary};

var gaussModel = function (observeSet, iterate, initValues) {
  
  var observeX = ee.Array(ee.List(observeSet).get(0)).pow(2).toList();
  observeSet = ee.List(observeSet).set(0, observeX);
  
  initValues = ee.List(initValues).set(1, ee.Number(1).divide(ee.Number(ee.List(initValues).get(1)).pow(2)));
  
  var coeff = OLSexponent(observeSet, iterate, initValues);
  
  coeff = ee.List(coeff).set(1, ee.Number(ee.List(coeff).get(1)).pow(-1/2));
  
  var f_model = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return ee.Number(ee.List(coeff).get(0)).multiply(ee.Number(1).subtract(((ee.Number(x).divide(ee.List(coeff).get(1))).pow(2).multiply(-1)).exp()));
  });
  
  var error = rmse(ee.List(observeSet).get(1), f_model);
  coeff = ee.List(coeff).add(error);

  return coeff;

};


// d) spherModel: The Spherical Semivariogram Model: y(h) = c0*[(3/2)*(h/a0) - (1/2)*(h/a0)^3];

// observeSet {list} - list of observing variable;

// return: Dictionary of coefficients for this model; {Dictionary};

var spherModel = function(observeSet) {
  
  var powerPolinom = 3;
  var zeroCoeff = ([0, 2]);
  
  var observeX = ee.List(ee.List(observeSet).get(0));
  var observeY = ee.List(ee.List(observeSet).get(1));
  
  var variants = ee.List.sequence(0, ee.Number(observeX.length()).subtract(1));
  
  var coeff = variants.map(function (Nsubset) {
    
    var observeX_submodel = (observeX).slice(0, ee.Number(Nsubset).add(1));
    var observeY_submodel = (observeY).slice(0, ee.Number(Nsubset).add(1));
    var observeSubset = ee.List([observeX_submodel, observeY_submodel]);
    
    var coeff_submodel = OLSpolinom(observeSubset, powerPolinom, zeroCoeff);
    
    var c0 = ee.Number(ee.List(coeff_submodel).get(1)).pow(3).multiply(-4).divide(ee.Number(ee.List(coeff_submodel).get(3)).multiply(27)).sqrt();
    var a0 = ee.Number(ee.List(coeff_submodel).get(1)).multiply(-1).divide(ee.Number(ee.List(coeff_submodel).get(3)).multiply(3)).sqrt();
    
    var tolerance = ee.Algorithms.If(ee.Number(Nsubset).eq(ee.Number(observeX.length()).subtract(1)), ee.Number(observeX.get(-1)), observeX.get(ee.Number(Nsubset).add(1)));
    a0 = ee.Algorithms.If(ee.Number(a0).lt(ee.Number(tolerance)), a0, null);
    
    return ee.List([Nsubset, c0, a0]);
    
  });
  
  var coeff_otbor = coeff.iterate(function (block, list) {
    return ee.Algorithms.If(ee.List(block).containsAll([null]), ee.List(list).remove(block), list);
  }, coeff);
  
  var coeff_error = ee.List(coeff_otbor).map(function (block) {
    
    var number = ee.List(block).get(0);
    
    var observeX_submodel1 = observeX.slice(0, ee.Number(number).add(1));
    var observeY_submodel1 = observeY.slice(0, ee.Number(number).add(1));
    var observeX_submodel2 = observeX.slice(number, observeX.length());
    var observeY_submodel2 = observeY.slice(number, observeX.length());
    
    var a0 = ee.Number(ee.List(block).get(2));
    var c0 = ee.Number(ee.List(block).get(1));
    
    var f_submodel1 = observeX_submodel1.map(function (x) {
      return c0.multiply(ee.Number(x).divide(a0).multiply(3).divide(2).subtract(ee.Number(x).divide(a0).pow(3).divide(2)));
    });
    
    var f_submodel2 = ee.List.repeat(c0, observeY_submodel2.length());
    
    var error_submodel1 = rmse(observeY_submodel1, f_submodel1);
    var error_submodel2 = rmse(observeY_submodel2, f_submodel2);
    var error = (ee.Number(error_submodel1).pow(2)).add(ee.Number(error_submodel2).pow(2)).sqrt();    
    
    return ee.List([error, number, c0, a0]);
    
  });  
  
  var error_min = ee.List(ee.Array(coeff_error).matrixTranspose(0, 1).toList().get(0)).reduce(ee.Reducer.min());
  var index = ee.List(ee.Array(coeff_error).matrixTranspose(0, 1).toList().get(0)).indexOf(error_min);
  var coeff_ = ee.List(coeff_error.slice(index, ee.Number(index).add(1)).get(0));
  
  return ee.List([coeff_.get(2), coeff_.get(3), coeff_.get(0)]);

};


// e) powExpModel: The combine of power and exponent model: y(h) = c0*(1 - exp(-x*a0) + d0*x^b0;

// observeSet {list} - list of observing variable;
// iterate {number}  - counts of repeating iterating process;
// initValues {list} - initial values for iterating process;

// return: Dictionary of coefficients for this model; {Dictionary};

var powExpModel = function (observeSet, iterate, initValues) {
  
  observeSet = noZero(observeSet);
  
  var PowerPolinom = 1;
  var zeroCoeff = ee.List([0]);
  
  var model_pow = powerModel(observeSet);
  var model_exp = exponentModel(observeSet, iterate, initValues);
  
  var a0 = ee.Number(ee.List(model_exp).get(1));
  var b0 = ee.Number(ee.List(model_pow).get(1));
  var c0 = ee.Number(ee.List(model_exp).get(0));
  var d0 = ee.Number(ee.List(model_pow).get(0));
  
  var pow = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return d0.multiply(ee.Number(x).pow(b0));
  });
  
  var exp = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return c0.multiply(ee.Number(1).subtract((ee.Number(x).multiply(-1).divide(a0)).exp()));
  }); 
    
  var observeX_new = ee.Array(exp).subtract(pow).toList();
  var observeY_new = ee.Array(ee.List(observeSet).get(1)).subtract(pow).toList();
  
  var observeSet_new = ee.List([observeX_new, observeY_new]);
  
  var coeff = OLSpolinom(observeSet_new, PowerPolinom, zeroCoeff);
  
  var alfa = ee.List(coeff).get(1);
  
  var f_model = ee.List(ee.List(observeSet).get(0)).map(function (x) {
    return ee.Number(alfa).multiply(c0.multiply(ee.Number(1).subtract(ee.Number(x).multiply(-1).divide(a0).exp())))
    .add((ee.Number(x).pow(b0).multiply(d0)).multiply(ee.Number(1).subtract(alfa)));
  });
  
  var error = rmse(ee.List(observeSet).get(1), f_model);
  
  return ee.List([a0, b0, c0, d0, error]);
  
};